In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
taz = gpd.read_file('./tazshapefile_909_TAZ/taz909.shp')
# taz = taz.to_crs('epsg:4326')

In [ ]:
from shapely.geometry import Point

df2 = gpd.read_file('./transit_demand.csv', crs="EPSG:4326" ).drop('geometry', axis=1)
df2['from_lat'] = df2.from_location.apply(lambda x: float(x.split(',')[0].split('(')[1]))
df2['from_lon'] = df2.from_location.apply(lambda x: float(x.split(',')[1].split(')')[0]))
df2['to_lat'] = df2.to_location.apply(lambda x: float(x.split(',')[0].split('(')[1]))
df2['to_lon'] = df2.to_location.apply(lambda x: float(x.split(',')[1].split(')')[0]))
from_geometry = [Point(xy) for xy in zip(df2['from_lat'], df2['from_lon'])]
to_geometry = [Point(xy) for xy in zip(df2['to_lat'], df2['to_lon'])]

In [ ]:
df2 = df2.reset_index()

In [ ]:
df2_from = gpd.GeoDataFrame(df2, crs="EPSG:4019", geometry=from_geometry).drop('to_location', axis=1)
df2_to = gpd.GeoDataFrame(df2, crs="EPSG:4019", geometry=to_geometry).drop('from_location', axis=1)

In [ ]:
df2_from = gpd.sjoin(df2_from, taz[['ID', 'geometry']], how='inner').rename(columns={'ID':'from_TAZ', 'geometry':'from_geom'})
df2_to = gpd.sjoin(df2_to, taz[['ID', 'geometry']], how='inner').rename(columns={'ID':'to_TAZ', 'geometry':'to_geom'})

In [ ]:
df2_final = df2_from.merge(df2_to[['index', 'person_id', 'to_location', 'to_TAZ', 'to_geom']], on=['index', 'person_id'], how='left')

In [ ]:
# df2_final.dropna()

In [ ]:
df2_final.drop(['index_right', 'from_lat', 'from_lon', 'to_lat', 'to_lon'], axis=1)[['person_id', 'from_geom', 'to_geom', 'from_TAZ', 'to_TAZ', 'mode', 'depart', 'sequence_idx']].to_csv('person_movement_TAZ.csv', index=False)